# Proyecto Final Inteligencia Artificial
## Universidad Técnica Ambato

**Integrantes:** Ismael Sailema - William Chimborazo


**Curso:** 7mo Software

# 1. Importaciones necesarias

In [1]:
import cv2
import os
import numpy as np

# 2. Verficación de videos disponibles

In [2]:
from pathlib import Path

# Verificar que la carpeta de videos existe y contiene videos
video_base = 'TrainingData/videos'
video_base_path = Path(video_base)

print("="*70)
print("DIAGNÓSTICO DE VIDEOS")
print("="*70)

if not video_base_path.exists():
    print(f"Carpeta no existe: {video_base}")
else:
    print(f"Carpeta encontrada: {video_base}")
    
    # Listar carpetas de personas
    person_dirs = sorted([d for d in video_base_path.iterdir() if d.is_dir()])
    print(f"Personas encontradas: {len(person_dirs)}")
    
    for person_dir in person_dirs:
        videos = list(person_dir.glob('*.mp4')) + list(person_dir.glob('*.avi'))
        print(f"  - {person_dir.name}: {len(videos)} videos")
        for vid in videos[:2]:
            print(f"      {vid.name}")


DIAGNÓSTICO DE VIDEOS
Carpeta encontrada: TrainingData/videos
Personas encontradas: 4
  - AlisonSalas: 2 videos
      Alison_video_1.mp4
      Alison_video_2.mp4
  - FreddyAlvarez: 3 videos
      Freddy_video_01.mp4
      Freddy_video_02.mp4
  - IsmaelSailema: 3 videos
      Ismael_video_01.mp4
      Ismael_video_02.mp4
  - WilliamChimborazo: 2 videos
      William_video_1.mp4
      William_video_2.mp4


# 3. Extracción de Frames

    Extrae frames de videos organizados por persona usando OpenCV (cv2)
    Guarda los frames en carpetas con nombres de personas
    
    Args:
        video_base_folder: Carpeta raíz con subcarpetas de personas
        output_base_folder: Carpeta raíz donde guardar frames
        fps: Frames por segundo a extraer


In [3]:
# Función para extraer frames de videos usando cv2
def extract_frames_from_videos(video_base_folder, output_base_folder, fps=2):
    from pathlib import Path
    
    video_base = Path(video_base_folder)
    output_base = Path(output_base_folder)
    output_base.mkdir(parents=True, exist_ok=True)
    
    # Iterar por carpetas de personas
    person_dirs = sorted([d for d in video_base.iterdir() if d.is_dir()])
    
    print(f"\n{'='*70}")
    print(f"EXTRAYENDO FRAMES DE {len(person_dirs)} PERSONA(S)")
    print(f"{'='*70}\n")
    
    total_videos = 0
    total_frames = 0
    
    for person_dir in person_dirs:
        person_name = person_dir.name
        print(f"Procesando: {person_name}")
        
        # Encontrar videos
        video_files = list(person_dir.glob('*.mp4')) + list(person_dir.glob('*.avi')) + list(person_dir.glob('*.mov'))
        
        if not video_files:
            print(f"  No se encontraron videos")
            continue
        
        for video_idx, video_file in enumerate(video_files, 1):
            # Crear carpeta con nombre de la persona
            person_output = output_base / person_name
            person_output.mkdir(parents=True, exist_ok=True)
            
            print(f"  Video {video_idx}: {video_file.name}")
            
            # Abrir video con cv2
            cap = cv2.VideoCapture(str(video_file))
            
            if not cap.isOpened():
                print(f"Error: No se pudo abrir el video")
                continue
            
            # Obtener FPS del video original
            video_fps = cap.get(cv2.CAP_PROP_FPS)
            if video_fps == 0:
                video_fps = 30  # FPS por defecto si no se puede obtener
            
            # Calcular cada cuántos frames extraer
            frame_interval = int(video_fps / fps)
            if frame_interval < 1:
                frame_interval = 1
            
            frame_count = 0
            saved_count = 0
            
            while True:
                ret, frame = cap.read()
                
                if not ret:
                    break
                
                # Guardar solo cada N frames según el FPS deseado
                if frame_count % frame_interval == 0:
                    output_filename = f'{person_name}_video{video_idx}_frame_{saved_count:04d}.png'
                    output_path = person_output / output_filename
                    cv2.imwrite(str(output_path), frame)
                    saved_count += 1
                
                frame_count += 1
            
            cap.release()
            
            if saved_count > 0:
                print(f"{saved_count} frames extraídos")
                total_frames += saved_count
                total_videos += 1
            else:
                print(f"No se extrajeron frames")
    
    print(f"\n{'='*70}")
    print(f"RESUMEN DE EXTRACCIÓN:")
    print(f"  Total de videos procesados: {total_videos}")
    print(f"  Total de frames extraídos: {total_frames}")
    print(f"  Método utilizado: OpenCV (cv2)")
    print(f"{'='*70}\n")


In [4]:
# Extraer frames de todos los videos
extract_frames_from_videos(
    'TrainingData/videos',
    'TrainingData/frames',
    fps=5
)


EXTRAYENDO FRAMES DE 4 PERSONA(S)

Procesando: AlisonSalas
  Video 1: Alison_video_1.mp4
430 frames extraídos
  Video 2: Alison_video_2.mp4
430 frames extraídos
  Video 2: Alison_video_2.mp4
120 frames extraídos
Procesando: FreddyAlvarez
  Video 1: Freddy_video_01.mp4
120 frames extraídos
Procesando: FreddyAlvarez
  Video 1: Freddy_video_01.mp4
260 frames extraídos
  Video 2: Freddy_video_02.mp4
260 frames extraídos
  Video 2: Freddy_video_02.mp4
104 frames extraídos
  Video 3: Freddy_video_03.mp4
104 frames extraídos
  Video 3: Freddy_video_03.mp4
129 frames extraídos
Procesando: IsmaelSailema
  Video 1: Ismael_video_01.mp4
129 frames extraídos
Procesando: IsmaelSailema
  Video 1: Ismael_video_01.mp4
129 frames extraídos
  Video 2: Ismael_video_02.mp4
129 frames extraídos
  Video 2: Ismael_video_02.mp4
212 frames extraídos
  Video 3: Ismael_video_3.mp4
212 frames extraídos
  Video 3: Ismael_video_3.mp4
285 frames extraídos
Procesando: WilliamChimborazo
  Video 1: William_video_1.mp4


# 4. Procesamiento de imágenes para FaceNet

**IMPORTANTE:** Las imágenes se guardan en **RGB 160x160** (formato requerido por FaceNet)
- RGB mantiene información de color (mejor que grayscale)
- 160x160 es el tamaño de entrada estándar de FaceNet
- Detección con Haar Cascade + validación múltiple


In [5]:
import face_recognition
from pathlib import Path
from PIL import Image
import shutil
import subprocess

# Rutas
frames_base = Path('TrainingData/frames')
faces_base = Path('TrainingData/faces')

# Limpiar carpeta de caras si existe
if faces_base.exists():
    shutil.rmtree(faces_base)
    print(f"Carpeta de caras eliminada\n")

faces_base.mkdir(parents=True, exist_ok=True)

# múltiples cascades para mejor detección
cascade_frontal = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cascade_profile = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')
cascade_eye = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

def is_valid_face(face_crop, top, right, bottom, left):
    """
    Valida si la región detectada es realmente un rostro
    Usa múltiples criterios: proporción, ojos, tamaño
    """
    # 1. Verificar tamaño mínimo razonable
    width = right - left
    height = bottom - top
    
    if width < 40 or height < 40:
        return False
    
    # 2. Verificar proporción (los rostros tienen proporción ~1:1.2)
    aspect_ratio = width / height
    if aspect_ratio < 0.6 or aspect_ratio > 1.5:
        return False
    
    # 3. Detectar ojos dentro de la región del rostro (validación fuerte)
    gray_face = cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)
    
    # Buscar ojos en la mitad superior del rostro
    upper_half = gray_face[0:int(height*0.6), :]
    eyes = cascade_eye.detectMultiScale(
        upper_half,
        scaleFactor=1.1,
        minNeighbors=3,
        minSize=(10, 10)
    )
    
    # Si no detecta al menos 1 ojo, probablemente no es un rostro
    if len(eyes) < 1:
        return False
    
    # 4. Verificar varianza de píxeles (evitar regiones muy uniformes)
    variance = np.var(gray_face)
    if variance < 100:  # Muy uniforme, probablemente no es un rostro
        return False
    
    return True

def detect_faces_improved(image):
    """
    Detecta caras usando Haar Cascade mejorado con validaciones múltiples
    Retorna lista de ubicaciones validadas en formato (top, right, bottom, left)
    """
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Aplicar ecualización adaptativa de histograma para mejorar contraste
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        gray_enhanced = clahe.apply(gray)
        
        # Detectar rostros frontales con parámetros MÁS ESTRICTOS
        faces_frontal = cascade_frontal.detectMultiScale(
            gray_enhanced,
            scaleFactor=1.05,      # Más preciso (antes 1.1)
            minNeighbors=8,        # Más estricto (antes 5)
            minSize=(60, 60),      # Tamaño mínimo mayor (antes 20x20)
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        
        # También detectar perfiles (opcional, pero útil)
        faces_profile = cascade_profile.detectMultiScale(
            gray_enhanced,
            scaleFactor=1.05,
            minNeighbors=8,
            minSize=(60, 60)
        )
        
        # Combinar detecciones
        all_faces = []
        
        # Procesar rostros frontales
        for (x, y, w, h) in faces_frontal:
            all_faces.append((x, y, w, h))
        
        # Procesar perfiles
        for (x, y, w, h) in faces_profile:
            all_faces.append((x, y, w, h))
        
        # Eliminar duplicados (caras detectadas dos veces)
        unique_faces = []
        for face in all_faces:
            x, y, w, h = face
            is_duplicate = False
            
            for existing_face in unique_faces:
                ex, ey, ew, eh = existing_face
                # Calcular solapamiento
                overlap_x = max(0, min(x + w, ex + ew) - max(x, ex))
                overlap_y = max(0, min(y + h, ey + eh) - max(y, ey))
                overlap_area = overlap_x * overlap_y
                
                if overlap_area > (w * h * 0.5):  # 50% de solapamiento
                    is_duplicate = True
                    break
            
            if not is_duplicate:
                unique_faces.append(face)
        
        # Validar cada rostro detectado
        validated_faces = []
        for (x, y, w, h) in unique_faces:
            # Convertir formato Haar (x,y,w,h) a (top,right,bottom,left)
            top, right, bottom, left = y, x + w, y + h, x
            
            # Extraer región del rostro para validación
            face_crop = image[top:bottom, left:right]
            
            # Validar si es realmente un rostro
            if is_valid_face(face_crop, top, right, bottom, left):
                validated_faces.append((top, right, bottom, left))
        
        return validated_faces
        
    except Exception as e:
        print(f"Error en detección: {e}")
        return []

print("="*70)
print("PROCESANDO FRAMES Y EXTRAYENDO CARAS (RGB 160x160 para FaceNet)")
print("="*70 + "\n")

total_frames = 0
total_faces = 0
total_rejected = 0

# Iterar por carpetas de personas
for person_folder in sorted(frames_base.iterdir()):
    if not person_folder.is_dir():
        continue
    
    person_name = person_folder.name
    print(f"Procesando: {person_name}")
    
    # Crear carpeta para las caras de esta persona
    person_faces_folder = faces_base / person_name
    person_faces_folder.mkdir(parents=True, exist_ok=True)
    
    face_count = 0
    rejected_count = 0
    frame_files = sorted(list(person_folder.glob('*.png')))
    
    for frame_file in frame_files:
        total_frames += 1
        
        try:
            # Cargar imagen en RGB (OpenCV carga en BGR, así que convertimos)
            image = cv2.imread(str(frame_file))
            
            if image is None:
                continue
            
            # Detectar caras con detección mejorada
            face_locations = detect_faces_improved(image)
            
            # Procesar cada cara detectada y validada
            for idx, (top, right, bottom, left) in enumerate(face_locations):
                # Agregar padding (15 píxeles para mejor contexto)
                padding = 15
                top = max(0, top - padding)
                left = max(0, left - padding)
                bottom = min(image.shape[0], bottom + padding)
                right = min(image.shape[1], right + padding)
                
                # Recortar la cara (mantener RGB)
                face_crop = image[top:bottom, left:right]
                
                # Verificar que tiene tamaño mínimo adecuado
                if face_crop.shape[0] < 60 or face_crop.shape[1] < 60:
                    rejected_count += 1
                    continue
                
                # Convertir de BGR (OpenCV) a RGB (FaceNet)
                face_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
                
                # Redimensionar a 160x160 (tamaño requerido por FaceNet)
                face_resized = cv2.resize(face_rgb, (160, 160), interpolation=cv2.INTER_LANCZOS4)
                
                # Convertir de vuelta a BGR para guardar con OpenCV
                face_bgr = cv2.cvtColor(face_resized, cv2.COLOR_RGB2BGR)
                
                # Guardar cara con naming: {person_name}_face_{id}
                output_filename = f'{person_name}_face_{face_count}.png'
                output_path = person_faces_folder / output_filename
                
                cv2.imwrite(str(output_path), face_bgr)
                face_count += 1
                total_faces += 1
        
        except Exception as e:
            continue
    
    total_rejected += rejected_count
    print(f"  {face_count} caras válidas extraídas | {rejected_count} rechazadas")

print("\n" + "="*70)
print("RESUMEN DE PROCESAMIENTO:")
print(f"  Total de frames procesados: {total_frames}")
print(f"  Total de caras extraídas: {total_faces}")
print(f"  Total de detecciones rechazadas: {total_rejected}")
print(f"  Tasa de precisión: {(total_faces/(total_faces+total_rejected)*100):.1f}%" if (total_faces+total_rejected) > 0 else "N/A")
print(f"  Método utilizado: Haar Cascade + Validación múltiple")
print(f"  Validaciones: Proporción, Ojos, Varianza, Tamaño")
print(f"  Formato: RGB 160x160 (compatible con FaceNet)")
print(f"  Carpeta de caras: {faces_base}")
print(f"  Formato de nombres: {{persona}}_face_{{id}}.png")
print("="*70)


c:\Users\User\Desktop\Projects\ProyectoFinalIA\envPROJ\Lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


Carpeta de caras eliminada

PROCESANDO FRAMES Y EXTRAYENDO CARAS (RGB 160x160 para FaceNet)

Procesando: AlisonSalas
  460 caras válidas extraídas | 0 rechazadas
Procesando: FreddyAlvarez
  460 caras válidas extraídas | 0 rechazadas
Procesando: FreddyAlvarez
  432 caras válidas extraídas | 0 rechazadas
Procesando: IsmaelSailema
  432 caras válidas extraídas | 0 rechazadas
Procesando: IsmaelSailema
  429 caras válidas extraídas | 0 rechazadas
Procesando: WilliamChimborazo
  429 caras válidas extraídas | 0 rechazadas
Procesando: WilliamChimborazo
  456 caras válidas extraídas | 0 rechazadas

RESUMEN DE PROCESAMIENTO:
  Total de frames procesados: 2248
  Total de caras extraídas: 1777
  Total de detecciones rechazadas: 0
  Tasa de precisión: 100.0%
  Método utilizado: Haar Cascade + Validación múltiple
  Validaciones: Proporción, Ojos, Varianza, Tamaño
  Formato: RGB 160x160 (compatible con FaceNet)
  Carpeta de caras: TrainingData\faces
  Formato de nombres: {persona}_face_{id}.png
  456

# 5. Balanceo de dataset

In [6]:
import random
from pathlib import Path
import shutil

def balance_face_dataset(faces_folder, method='undersample', target_count=None, augment=True):
    """
    Equilibra el dataset de rostros para que todas las personas tengan número similar de imágenes
    
    Args:
        faces_folder: Carpeta con subcarpetas de rostros por persona
        method: 'undersample' (reducir), 'oversample' (aumentar) o 'hybrid' (combinado)
        target_count: Número objetivo de imágenes por persona (None = automático)
        augment: Si True, usa data augmentation al hacer oversample
    
    Returns:
        dict con estadísticas del balanceo
    """
    faces_base = Path(faces_folder)
    
    # Contar imágenes por persona
    person_counts = {}
    for person_folder in sorted(faces_base.iterdir()):
        if not person_folder.is_dir():
            continue
        
        face_files = list(person_folder.glob('*.png'))
        person_counts[person_folder.name] = {
            'folder': person_folder,
            'files': face_files,
            'count': len(face_files)
        }
    
    if not person_counts:
        print("No se encontraron carpetas de personas")
        return None
    
    # Calcular estadísticas
    counts = [p['count'] for p in person_counts.values()]
    min_count = min(counts)
    max_count = max(counts)
    avg_count = sum(counts) // len(counts)
    
    print("="*70)
    print("ANÁLISIS DEL DATASET")
    print("="*70)
    print(f"Personas: {len(person_counts)}")
    print(f"Mínimo de imágenes: {min_count}")
    print(f"Máximo de imágenes: {max_count}")
    print(f"Promedio: {avg_count}")
    print(f"Desbalance: {max_count - min_count} imágenes de diferencia")
    print("\nDistribución por persona:")
    for name, data in sorted(person_counts.items(), key=lambda x: x[1]['count'], reverse=True):
        print(f"  {name:20s}: {data['count']:3d} imágenes ")
    
    # Determinar target_count según el método
    if target_count is None:
        if method == 'undersample':
            target_count = min_count
        elif method == 'oversample':
            target_count = max_count
        else:  # hybrid
            target_count = avg_count
    
    print(f"\n{'='*70}")
    print(f"MÉTODO: {method.upper()} | OBJETIVO: {target_count} imágenes por persona")
    print("="*70 + "\n")
    
    stats = {
        'before': person_counts.copy(),
        'after': {},
        'removed': 0,
        'created': 0
    }
    
    # Aplicar balanceo
    for person_name, data in person_counts.items():
        current_count = data['count']
        person_folder = data['folder']
        files = data['files']
        
        if current_count == target_count:
            print(f"{person_name}: Ya tiene {target_count} imágenes")
            stats['after'][person_name] = current_count
            continue
        
        if current_count > target_count:
            # REDUCIR: Eliminar imágenes aleatorias
            to_remove = current_count - target_count
            files_to_remove = random.sample(files, to_remove)
            
            for file_path in files_to_remove:
                file_path.unlink()
                stats['removed'] += 1
            
            remaining = current_count - to_remove
            stats['after'][person_name] = remaining
            print(f"{person_name}: Reducido de {current_count} → {remaining} (-{to_remove})")
        
        else:
            # AUMENTAR: Duplicar o aumentar con augmentation
            to_add = target_count - current_count
            
            # Obtener el máximo número de archivo existente para continuar la numeración
            existing_numbers = []
            for f in files:
                # Extraer número del nombre: {person_name}_face_{num}.png
                stem = f.stem  # nombre sin extensión
                parts = stem.split('_')
                if len(parts) >= 3:
                    try:
                        num = int(parts[-1])
                        existing_numbers.append(num)
                    except ValueError:
                        pass
            
            # Comenzar desde el siguiente número disponible
            next_face_id = max(existing_numbers) + 1 if existing_numbers else current_count
            
            if augment:
                # Data Augmentation: rotaciones, flips, brillo (mantener RGB)
                added = 0
                while added < to_add:
                    # Seleccionar imagen aleatoria para aumentar
                    source_file = random.choice(files)
                    # Cargar en color (RGB)
                    img = cv2.imread(str(source_file), cv2.IMREAD_COLOR)
                    
                    if img is None:
                        continue
                    
                    # Aplicar transformación aleatoria
                    transform_type = random.choice(['rotate', 'flip', 'brightness', 'contrast'])
                    
                    if transform_type == 'rotate':
                        # Rotar entre -15 y 15 grados
                        angle = random.uniform(-15, 15)
                        h, w = img.shape[:2]
                        M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
                        img_aug = cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_REPLICATE)
                    
                    elif transform_type == 'flip':
                        # Flip horizontal
                        img_aug = cv2.flip(img, 1)
                    
                    elif transform_type == 'brightness':
                        # Ajustar brillo
                        beta = random.randint(-30, 30)
                        img_aug = cv2.convertScaleAbs(img, alpha=1.0, beta=beta)
                    
                    else:  # contrast
                        # Ajustar contraste
                        alpha = random.uniform(0.8, 1.2)
                        img_aug = cv2.convertScaleAbs(img, alpha=alpha, beta=0)
                    
                    # Guardar imagen aumentada con el mismo formato: {person_name}_face_{id}.png
                    new_filename = f"{person_name}_face_{next_face_id}.png"
                    new_path = person_folder / new_filename
                    cv2.imwrite(str(new_path), img_aug)
                    next_face_id += 1
                    added += 1
                    stats['created'] += 1
                
                final_count = current_count + added
                stats['after'][person_name] = final_count
                print(f"{person_name}: Aumentado de {current_count} → {final_count} (+{added} con augmentation)")
            
            else:
                # Simple duplicación con el mismo formato
                added = 0
                while added < to_add:
                    source_file = random.choice(files)
                    new_filename = f"{person_name}_face_{next_face_id}.png"
                    new_path = person_folder / new_filename
                    shutil.copy2(source_file, new_path)
                    next_face_id += 1
                    added += 1
                    stats['created'] += 1
                
                final_count = current_count + added
                stats['after'][person_name] = final_count
                print(f"{person_name}: Aumentado de {current_count} → {final_count} (+{added} duplicados)")
    
    # Resumen final
    print("\n" + "="*70)
    print("RESUMEN DEL BALANCEO:")
    print(f"  Imágenes eliminadas: {stats['removed']}")
    print(f"  Imágenes creadas: {stats['created']}")
    print(f"  Balance final: {target_count} imágenes por persona")
    print(f"  Dataset balanceado: ✓ COMPLETO")
    print("="*70)
    
    return stats


In [7]:

balance_stats = balance_face_dataset(
    faces_folder='TrainingData/faces',
    method='hybrid',          
    target_count=None,         # None = automático, o especifica un número como 100
    augment=True               # True = usa data augmentation, False = duplica imágenes
)


ANÁLISIS DEL DATASET
Personas: 4
Mínimo de imágenes: 429
Máximo de imágenes: 460
Promedio: 444
Desbalance: 31 imágenes de diferencia

Distribución por persona:
  AlisonSalas         : 460 imágenes 
  WilliamChimborazo   : 456 imágenes 
  FreddyAlvarez       : 432 imágenes 
  IsmaelSailema       : 429 imágenes 

MÉTODO: HYBRID | OBJETIVO: 444 imágenes por persona

AlisonSalas: Reducido de 460 → 444 (-16)
FreddyAlvarez: Aumentado de 432 → 444 (+12 con augmentation)
IsmaelSailema: Aumentado de 429 → 444 (+15 con augmentation)
WilliamChimborazo: Reducido de 456 → 444 (-12)

RESUMEN DEL BALANCEO:
  Imágenes eliminadas: 28
  Imágenes creadas: 27
  Balance final: 444 imágenes por persona
  Dataset balanceado: ✓ COMPLETO


# Verificar

In [8]:
# Verificar el balance final del dataset
from pathlib import Path

faces_base = Path('TrainingData/faces')

print("="*70)
print("VERIFICACIÓN FINAL DEL DATASET")
print("="*70 + "\n")

person_counts = {}
for person_folder in sorted(faces_base.iterdir()):
    if not person_folder.is_dir():
        continue
    
    face_count = len(list(person_folder.glob('*.png')))
    person_counts[person_folder.name] = face_count

if person_counts:
    counts = list(person_counts.values())
    min_count = min(counts)
    max_count = max(counts)
    avg_count = sum(counts) / len(counts)
    
    print(f"Total de personas: {len(person_counts)}")
    print(f"Total de imágenes: {sum(counts)}")
    print(f"Promedio por persona: {avg_count:.1f}")
    print(f"Rango: {min_count} - {max_count}")
    print(f"Desviación: {max_count - min_count}")
    
    if max_count - min_count <= 5:
        print(f"\nDataset PERFECTAMENTE equilibrado (diferencia ≤ 5)")
    elif max_count - min_count <= 15:
        print(f"\nDataset BIEN equilibrado (diferencia ≤ 15)")
    else:
        print(f"\nDataset con desbalance (diferencia > 15)")
    
    print("\nDistribución final:")
    for name, count in sorted(person_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"  {name:20s}: {count:3d} imágenes")
    
    print("\n" + "="*70)
    print("Dataset listo para entrenamiento del modelo")
    print("="*70)
else:
    print("No se encontraron carpetas de rostros")


VERIFICACIÓN FINAL DEL DATASET

Total de personas: 4
Total de imágenes: 1776
Promedio por persona: 444.0
Rango: 444 - 444
Desviación: 0

Dataset PERFECTAMENTE equilibrado (diferencia ≤ 5)

Distribución final:
  AlisonSalas         : 444 imágenes
  FreddyAlvarez       : 444 imágenes
  IsmaelSailema       : 444 imágenes
  WilliamChimborazo   : 444 imágenes

Dataset listo para entrenamiento del modelo
